# Train-Test

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd

import os
import time
import gc

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.ensemble import ExtraTreesClassifier

from lightgbm import LGBMClassifier

import random

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F

Mounted at /content/drive


In [2]:
test=pd.read_csv('/content/drive/MyDrive/data/result_embed.csv')
test.head()

,ID,Family,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,01kcPWA9K2BOxQeS5Rju,1,-0.338611,1.291727,0.118837,-1.118286,0.183966,-0.405247,-0.729886,-0.332307,...,-0.475346,-0.425257,-0.000070,0.276955,0.192112,1.176439,-0.234972,-0.730219,1.071440,-0.091770
1,04EjIdbPV5e1XroFOpiN,1,-0.428330,1.376984,0.234024,-0.627847,0.213055,-0.609343,-0.659496,-0.320076,...,-0.408290,-0.389868,-0.164271,0.207200,0.035918,1.313510,-0.123089,-0.806177,1.062548,-0.169090
2,05EeG39MTRrI6VY21DPd,1,-0.377473,1.542212,0.202506,-0.368294,0.447894,-0.722350,-0.336275,-0.411114,...,-0.553859,-0.478773,-0.139812,0.208929,0.004945,1.640448,-0.340988,-0.576668,1.164637,-0.100652
3,05rJTUWYAKNegBk2wE8X,1,-0.354789,1.304047,0.117015,-0.903091,0.150693,-0.642379,-0.896521,-0.305303,...,-0.351606,-0.341613,-0.220712,0.312799,0.046395,1.294530,-0.131182,-0.718261,1.172608,-0.229033
4,0AnoOZDNbPXIr2MRBSCJ,1,-0.175248,1.296375,-0.155661,-0.533257,0.156963,-0.595596,-0.341953,-0.253579,...,-0.324669,-0.036423,-0.272495,-0.101691,0.137479,1.847042,-0.233596,-0.768067,0.979953,0.340067


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
X = test.drop(['ID', 'Family'], axis=1)
y = test['Family']
# Split the dataset into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Merge the features and labels back with ID and Family columns
train_set = pd.concat([test[['ID', 'Family']].iloc[X_train.index], X_train], axis=1)
test_set = pd.concat([test[['ID', 'Family']].iloc[X_test.index], X_test], axis=1)


In [12]:
#train_set=train_set.drop(['ID', 'Family'], axis=1)
train_set.head()

,ID,Family,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
9030,AGgJTSxpu6rR8mayOhDz,8,-0.041838,2.143785,0.314858,0.611004,0.656308,-0.581835,0.451072,-0.716000,...,-0.646717,-0.409990,-0.199381,-0.304473,-0.011557,2.183214,-0.049548,-1.503728,0.661782,0.545309
8873,6meYANGp0FbrZU1ou3jy,8,-0.317847,1.333996,0.017145,-0.837949,0.186298,-0.622943,-0.667178,-0.302923,...,-0.360639,-0.312786,-0.223538,0.198285,0.037862,1.388862,-0.193021,-0.746462,1.147938,-0.115610
1960,5FXu1yi9MvKsJIGPC72c,2,-0.181569,1.306641,-0.074673,-0.180636,0.221512,-0.480010,0.274712,-0.295886,...,-0.326704,-0.179272,-0.093445,-0.257934,0.188095,1.619035,-0.234538,-0.977880,0.917972,0.399178
8004,g4EyujTPMQHLfVIRlDWS,6,-0.334665,1.337375,-0.011870,-1.151497,0.284409,-0.627941,-0.879311,-0.364044,...,-0.476459,-0.433213,-0.165550,0.442951,0.044517,1.380580,-0.430455,-0.447597,1.352959,-0.181184
6280,gp7MxT3EYbicRlBKty90,3,-0.611482,1.646740,0.140485,-1.120854,0.776186,-0.252365,-0.020733,-0.538375,...,-0.821075,-0.655344,0.256858,0.231460,-0.097218,1.014031,-0.505972,-0.885432,1.272357,0.025069


In [13]:
#test_set=test_set.drop(['ID', 'Family'], axis=1)
test_set.head()

,ID,Family,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
2061,6ml5DH9TvnIiKReLuP2d,2,-0.200730,0.579599,-0.959288,-0.657794,-0.318916,0.019931,0.714088,-0.063379,...,0.396721,0.507734,0.248525,-1.061210,-0.039218,1.684787,-0.904391,-0.498062,1.417000,0.926538
8671,15uCdAByiF48LpZ3YS6Q,8,-0.018660,2.195822,0.343763,0.665885,0.706656,-0.600788,0.467918,-0.750159,...,-0.630975,-0.392268,-0.163800,-0.300104,-0.038024,2.180972,-0.019015,-1.533542,0.707606,0.500485
8286,5EVhC0IMi1dNb7TtgxRa,7,-0.954539,1.471527,0.285202,-1.603462,1.017479,-0.489029,-0.134877,-0.546498,...,-0.976189,-1.180292,0.385070,0.847393,-0.110100,0.557097,-0.974714,-0.193514,1.710362,-0.478235
5522,CoQn9bfkvMUjWwVgPAFL,3,-0.754701,1.604011,0.063157,-1.462972,1.162014,-0.591681,0.495901,-0.701966,...,-1.022316,-0.963337,0.232041,0.976257,0.154573,0.855935,-1.057585,-0.327719,0.871632,-0.067173
1835,3Oy7QgdVxkjuEsfXT8B5,2,-0.178179,1.379395,0.070312,-0.062838,0.340935,-0.484584,0.247614,-0.366754,...,-0.365241,-0.232123,-0.061479,-0.190999,0.139995,1.597580,-0.234619,-0.906047,0.867897,0.286408


In [19]:
# Assuming your test set DataFrame is named test_set
id_to_find = "3Oy7QgdVxkjuEsfXT8B5"

# Find the row with the specified ID in the test set
result_row = test[test['ID'] == id_to_find]
result_row_train = test_set[test_set['ID'] == id_to_find]

# Print the result
result_row

,ID,Family,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
1835,3Oy7QgdVxkjuEsfXT8B5,2,-0.178179,1.379395,0.070312,-0.062838,0.340935,-0.484584,0.247614,-0.366754,...,-0.365241,-0.232123,-0.061479,-0.190999,0.139995,1.59758,-0.234619,-0.906047,0.867897,0.286408


In [20]:
result_row_train

,ID,Family,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
1835,3Oy7QgdVxkjuEsfXT8B5,2,-0.178179,1.379395,0.070312,-0.062838,0.340935,-0.484584,0.247614,-0.366754,...,-0.365241,-0.232123,-0.061479,-0.190999,0.139995,1.59758,-0.234619,-0.906047,0.867897,0.286408


In [21]:
rows_equal = result_row.equals(result_row_train)

# Print the result
print(f"Rows are equal: {rows_equal}")

Rows are equal: True


In [30]:
len(train_set)

8694

In [25]:
train_set.to_csv('/content/drive/MyDrive/new_data/new_train_set.csv', index=False)

In [26]:
test_set.to_csv('/content/drive/MyDrive/new_data/new_test_set.csv', index=False)

In [31]:
train_set.head()

,ID,Family,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
9030,AGgJTSxpu6rR8mayOhDz,8,-0.041838,2.143785,0.314858,0.611004,0.656308,-0.581835,0.451072,-0.716000,...,-0.646717,-0.409990,-0.199381,-0.304473,-0.011557,2.183214,-0.049548,-1.503728,0.661782,0.545309
8873,6meYANGp0FbrZU1ou3jy,8,-0.317847,1.333996,0.017145,-0.837949,0.186298,-0.622943,-0.667178,-0.302923,...,-0.360639,-0.312786,-0.223538,0.198285,0.037862,1.388862,-0.193021,-0.746462,1.147938,-0.115610
1960,5FXu1yi9MvKsJIGPC72c,2,-0.181569,1.306641,-0.074673,-0.180636,0.221512,-0.480010,0.274712,-0.295886,...,-0.326704,-0.179272,-0.093445,-0.257934,0.188095,1.619035,-0.234538,-0.977880,0.917972,0.399178
8004,g4EyujTPMQHLfVIRlDWS,6,-0.334665,1.337375,-0.011870,-1.151497,0.284409,-0.627941,-0.879311,-0.364044,...,-0.476459,-0.433213,-0.165550,0.442951,0.044517,1.380580,-0.430455,-0.447597,1.352959,-0.181184
6280,gp7MxT3EYbicRlBKty90,3,-0.611482,1.646740,0.140485,-1.120854,0.776186,-0.252365,-0.020733,-0.538375,...,-0.821075,-0.655344,0.256858,0.231460,-0.097218,1.014031,-0.505972,-0.885432,1.272357,0.025069


In [27]:
old_op=pd.read_csv('/content/drive/MyDrive/data/dataset.csv')

In [28]:
import pandas as pd

# Assuming train_set and old_op DataFrames are already loaded
# Merge based on the 'ID' column
merged_data = pd.merge(old_op, train_set[['ID']], on='ID', how='inner')

# Print the resulting DataFrame
len(merged_data)


8694

In [33]:
merged_data= pd.DataFrame(train_set['ID']).merge(merged_data, on='ID').reset_index(drop = True)
merged_data.head()


,ID,Opcodes,Family
0,AGgJTSxpu6rR8mayOhDz,"push,mov,sub,mov,mov,mov,mov,sub,mov,cmp,jnz,m...",8
1,6meYANGp0FbrZU1ou3jy,"push,mov,push,mov,test,jz,mov,push,lea,push,pu...",8
2,5FXu1yi9MvKsJIGPC72c,"mov,jmp,push,mov,mov,call,test,jz,push,call,ad...",2
3,g4EyujTPMQHLfVIRlDWS,"sub,push,push,push,push,push,lea,push,call,tes...",6
4,gp7MxT3EYbicRlBKty90,"mov,push,mov,sub,lea,xor,push,push,push,push,c...",3


In [34]:
merged_data.to_csv('/content/drive/MyDrive/new_data/new_train_set_opcodes.csv', index=False)

In [35]:
import pandas as pd

# Assuming train_set and old_op DataFrames are already loaded
# Merge based on the 'ID' column
merged_data_test = pd.merge(old_op, test_set[['ID']], on='ID', how='inner')
merged_data_test= pd.DataFrame(test_set['ID']).merge(merged_data_test, on='ID').reset_index(drop = True)
merged_data_test.head()


,ID,Opcodes,Family
0,6ml5DH9TvnIiKReLuP2d,"xor,retn,push,mov,mov,mov,sub,push,push,push,p...",2
1,15uCdAByiF48LpZ3YS6Q,"push,mov,sub,mov,mov,mov,mov,sub,mov,cmp,jnz,m...",8
2,5EVhC0IMi1dNb7TtgxRa,"mov,push,push,push,push,push,call,pop,push,cal...",7
3,CoQn9bfkvMUjWwVgPAFL,"mov,push,mov,sub,mov,or,push,push,push,push,ca...",3
4,3Oy7QgdVxkjuEsfXT8B5,"mov,jmp,push,mov,mov,call,test,jz,push,call,ad...",2


In [36]:
len(merged_data_test)

2174

In [37]:
len(test_set)

2174

In [38]:
merged_data_test.to_csv('/content/drive/MyDrive/new_data/new_test_set_opcodes.csv', index=False)

# Appended Train-Test

In [39]:
old_appended_em=pd.read_csv('/content/drive/MyDrive/data/appended_result_embed.csv')
old_appended_em.head()

,ID,Family,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,01kcPWA9K2BOxQeS5Rju,1,-0.426271,1.495527,0.442969,-1.784287,0.687246,-0.163921,-0.821085,-0.767919,...,-0.816768,-1.232931,0.312494,0.956752,0.134952,0.596190,-0.748863,-0.692929,1.376552,-0.300369
1,04EjIdbPV5e1XroFOpiN,1,-0.429474,1.381561,0.241621,-0.654752,0.227177,-0.599185,-0.663230,-0.332880,...,-0.419219,-0.413807,-0.152669,0.227969,0.036689,1.294578,-0.139708,-0.803917,1.071521,-0.173929
2,05EeG39MTRrI6VY21DPd,1,-0.386531,1.551624,0.240367,-0.513145,0.496933,-0.666564,-0.377270,-0.464443,...,-0.594439,-0.584387,-0.087269,0.306046,0.010809,1.528025,-0.402952,-0.582781,1.199226,-0.128288
3,05rJTUWYAKNegBk2wE8X,1,-0.355359,1.305468,0.119359,-0.908725,0.154450,-0.639722,-0.896534,-0.308548,...,-0.354546,-0.347760,-0.217605,0.317562,0.046546,1.289890,-0.135289,-0.718038,1.174416,-0.230006
4,0AnoOZDNbPXIr2MRBSCJ,1,-0.193080,1.316853,-0.107708,-0.632932,0.209633,-0.560659,-0.372401,-0.302188,...,-0.367623,-0.139803,-0.225833,-0.011875,0.134646,1.751419,-0.285946,-0.762208,1.015983,0.295280


In [41]:
import pandas as pd

# Assuming train_set and old_op DataFrames are already loaded
# Merge based on the 'ID' column
merged_data_ap_train = pd.merge(old_appended_em, train_set[['ID']], on='ID', how='inner')
merged_data_ap_train = pd.DataFrame(train_set['ID']).merge(merged_data_ap_train , on='ID').reset_index(drop = True)
merged_data_ap_train .head()

,ID,Family,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,AGgJTSxpu6rR8mayOhDz,8,-0.086750,2.097701,0.354694,0.321065,0.701767,-0.520697,0.319220,-0.757801,...,-0.692105,-0.558416,-0.123065,-0.124055,-0.002078,1.979392,-0.161135,-1.421450,0.757271,0.445182
1,6meYANGp0FbrZU1ou3jy,8,-0.320289,1.338473,0.026504,-0.858128,0.198706,-0.614092,-0.670264,-0.314079,...,-0.370599,-0.334240,-0.212850,0.216131,0.038495,1.371703,-0.206279,-0.745325,1.154465,-0.120450
2,5FXu1yi9MvKsJIGPC72c,2,-0.211623,1.340872,0.000144,-0.385014,0.305977,-0.430808,0.164546,-0.375458,...,-0.400335,-0.343546,-0.030043,-0.088919,0.178478,1.476081,-0.324420,-0.948132,0.985708,0.316663
3,g4EyujTPMQHLfVIRlDWS,6,-0.337115,1.342268,-0.001111,-1.169186,0.296673,-0.618095,-0.879612,-0.375467,...,-0.485760,-0.455137,-0.154598,0.459073,0.045118,1.361756,-0.441617,-0.450717,1.357164,-0.185568
4,gp7MxT3EYbicRlBKty90,3,-0.587714,1.652140,0.266958,-1.389820,0.851282,-0.187895,-0.211603,-0.669924,...,-0.884184,-0.932247,0.327569,0.516444,-0.057535,0.814471,-0.655102,-0.836643,1.352212,-0.084716


In [42]:
len(merged_data_ap_train)

8694

In [43]:
merged_data_ap_train.to_csv('/content/drive/MyDrive/new_data/new_appended_train_set_embed.csv', index=False)

In [44]:
import pandas as pd

# Assuming train_set and old_op DataFrames are already loaded
# Merge based on the 'ID' column
merged_data_ap_test = pd.merge(old_appended_em, test_set[['ID']], on='ID', how='inner')
merged_data_ap_test = pd.DataFrame(test_set['ID']).merge(merged_data_ap_test , on='ID').reset_index(drop = True)
merged_data_ap_test.head()

,ID,Family,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,6ml5DH9TvnIiKReLuP2d,2,-0.273891,0.844779,-0.550763,-1.071082,0.031010,0.025918,0.322158,-0.325863,...,0.030408,-0.084421,0.329638,-0.428547,-0.008916,1.298694,-0.974443,-0.537559,1.471194,0.584834
1,15uCdAByiF48LpZ3YS6Q,8,-0.066754,2.143662,0.381500,0.364830,0.747991,-0.536572,0.331819,-0.789375,...,-0.678814,-0.545352,-0.089536,-0.116601,-0.025727,1.973398,-0.135816,-1.446693,0.800398,0.402865
2,5EVhC0IMi1dNb7TtgxRa,7,-0.775248,1.550799,0.458664,-1.903086,1.058921,-0.277613,-0.437969,-0.783340,...,-1.029900,-1.477625,0.463311,1.123425,-0.032578,0.373958,-1.061144,-0.378616,1.682552,-0.479172
3,CoQn9bfkvMUjWwVgPAFL,3,-0.701969,1.618118,0.200858,-1.650105,1.153681,-0.458905,0.204628,-0.794271,...,-1.040981,-1.165220,0.305186,1.094754,0.140173,0.697112,-1.085815,-0.397208,1.032011,-0.153447
4,3Oy7QgdVxkjuEsfXT8B5,2,-0.198431,1.397670,0.111131,-0.206482,0.389787,-0.451490,0.175851,-0.415380,...,-0.411928,-0.338354,-0.021202,-0.082488,0.136591,1.503601,-0.294547,-0.890704,0.916194,0.238440


In [45]:
len(merged_data_ap_test)

2174

In [46]:
merged_data_ap_test.to_csv('/content/drive/MyDrive/new_data/new_appended_test_set_embed.csv', index=False)

In [48]:
old_appended_op=pd.read_csv('/content/drive/MyDrive/data/appended_new_opcodes.csv')
old_appended_op.head()

,ID,Opcodes,Family
0,01kcPWA9K2BOxQeS5Rju,push push mov push mov sub mov push push lea m...,1
1,04EjIdbPV5e1XroFOpiN,push push mov push mov mov mov lea push retn m...,1
2,05EeG39MTRrI6VY21DPd,push mov mov test mov jz push call mov test jz...,1
3,05rJTUWYAKNegBk2wE8X,cmp jnz test jnz retn mov push mov mov push mo...,1
4,0AnoOZDNbPXIr2MRBSCJ,push mov sub mov and and push push mov cmp mov...,1


In [49]:
import pandas as pd

# Assuming train_set and old_op DataFrames are already loaded
# Merge based on the 'ID' column
merged_data_ap_train_op = pd.merge(old_appended_op, train_set[['ID']], on='ID', how='inner')
merged_data_ap_train_op = pd.DataFrame(train_set['ID']).merge(merged_data_ap_train_op, on='ID').reset_index(drop = True)
merged_data_ap_train_op.head()

,ID,Opcodes,Family
0,AGgJTSxpu6rR8mayOhDz,push mov sub mov mov mov mov sub mov cmp jnz m...,8
1,6meYANGp0FbrZU1ou3jy,push mov push mov test jz mov push lea push pu...,8
2,5FXu1yi9MvKsJIGPC72c,mov jmp push mov mov call test jz push call ad...,2
3,g4EyujTPMQHLfVIRlDWS,sub push push push push push lea push call tes...,6
4,gp7MxT3EYbicRlBKty90,mov push mov sub lea xor push push push push c...,3


In [50]:
len(merged_data_ap_train_op)

8694

In [51]:
merged_data_ap_train_op.to_csv('/content/drive/MyDrive/new_data/new_appended_train_set_opcodes.csv', index=False)

In [52]:
import pandas as pd

# Assuming train_set and old_op DataFrames are already loaded
# Merge based on the 'ID' column
merged_data_ap_test_op = pd.merge(old_appended_op, test_set[['ID']], on='ID', how='inner')
merged_data_ap_test_op = pd.DataFrame(test_set['ID']).merge(merged_data_ap_test_op, on='ID').reset_index(drop = True)
merged_data_ap_test_op.head()

,ID,Opcodes,Family
0,6ml5DH9TvnIiKReLuP2d,xor retn push mov mov mov sub push push push p...,2
1,15uCdAByiF48LpZ3YS6Q,push mov sub mov mov mov mov sub mov cmp jnz m...,8
2,5EVhC0IMi1dNb7TtgxRa,mov push push push push push call pop push cal...,7
3,CoQn9bfkvMUjWwVgPAFL,mov push mov sub mov or push push push push ca...,3
4,3Oy7QgdVxkjuEsfXT8B5,mov jmp push mov mov call test jz push call ad...,2


In [53]:
len(merged_data_ap_test_op)

2174

In [55]:
merged_data_ap_test_op.to_csv('/content/drive/MyDrive/new_data/new_appended_test_set_opcodes.csv', index=False)

In [56]:
import os

folder_path = '/content/drive/MyDrive/new_data'  # Replace with the actual path to your "new_data" folder

# List all files in the folder
file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Print the list of files
print("List of files in the new_data folder:")
for file_name in file_list:
    print(file_name)


List of files in the new_data folder:
new_train_set.csv
new_test_set.csv
new_train_set_opcodes.csv
new_test_set_opcodes.csv
new_appended_train_set_embed.csv
new_appended_test_set_embed.csv
new_appended_train_set_opcodes.csv
new_appended_test_set_opcodes.csv
